# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "../WeatherPy/citydata.csv"
cityweatherdf = pd.read_csv(filepath)
cityweatherdf.head()


,searc_city,city,country,lat,lng,temp,humidity,cloudiness,windspeed
0,ushuaia,Ushuaia,AR,-54.8000,-68.3000,50.00,53,40,20.71
1,oytal,Oytal,KZ,42.9057,73.2697,55.27,56,33,7.20
2,bengkulu,Bengkulu,ID,-3.8004,102.2655,77.02,91,99,3.87
3,muros,Muros,ES,42.7762,-9.0603,62.02,94,100,3.00
4,rock sound,Rock Sound,BS,24.9000,-76.2000,77.90,73,70,12.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cityweatherdf[["lat", "lng"]]

# Fill NaN values and convert to float
humid = cityweatherdf["humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=np.max(humid),
                                 point_radius=3.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Criteria is 
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
short_listdf = cityweatherdf.loc[(cityweatherdf["windspeed"] <= 10) & \
                               (cityweatherdf["cloudiness"] == 0) & \
                               (cityweatherdf["temp"].between(70,80))].dropna()


cityrecordcount = len(short_listdf.index)
print(f"Total cities meeting criteria : {cityrecordcount}")
short_listdf.head()

Total cities meeting criteria : 10


,searc_city,city,country,lat,lng,temp,humidity,cloudiness,windspeed
127,richards bay,Richards Bay,ZA,-28.7830,32.0377,73.18,36,0,4.68
129,santiago del estero,Santiago del Estero,AR,-27.7951,-64.2615,72.59,36,0,7.92
217,nador,Nador,MA,35.1740,-2.9287,75.87,60,0,5.75
254,zmiyiv,Zmiyiv,UA,49.6966,36.3611,73.20,38,0,8.95
299,komsomolskiy,Yangi Marg`ilon,UZ,40.4272,71.7189,71.60,37,0,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Process results and return top hotel
def tophotel(data):
    FIELDS = ["icon","name","photos","place_id","rating","geometry.location.lat","geometry.location.lng"]
    testhoteldf = pd.json_normalize(data)
    # Sort to find top rated hotel
    sorted_hoteldf = testhoteldf[FIELDS].sort_values(by=['rating'],ascending=False).reset_index(drop=True)
    sorted_hoteldf.rename(columns={"name":"Hotel_Name","geometry.location.lat":"Lat","geometry.location.lng":"Lng"}, inplace=True)

    return sorted_hoteldf.iloc[0]

# geocoordinates

def hotelsearch(lat,lng, allresults=False):  # Return Top Hote by default, set "allresults" true if you want complete return
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    target_coordinates = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    try :
        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        json_response = response.json()
        searchresults = json_response['results']
        if allresults:
            return response.json()
        else:
            if not searchresults:
                #print("No hotels returned!")
                return
            else:
                tophotelresult = tophotel(searchresults)
                return tophotelresult
    except requests.exceptions.RequestException as e:
        #Do not want to continue if getting errors
        raise SystemExit(e)


In [7]:
# Test Return of Top Hotel
# Atlanta is lat': 33.749, 'lng': -84.388,
atlanta = {"lat":'33.749',"lng":'-84.388'}
hoteltest = hotelsearch(atlanta["lat"],atlanta["lng"])
print(hoteltest)

icon          https://maps.gstatic.com/mapfiles/place_api/ic...
Hotel_Name                     Canopy By Hilton Atlanta Midtown
photos        [{'height': 472, 'html_attributions': ['<a hre...
place_id                            ChIJ477LYFAE9YgR24puh_com0g
rating                                                      4.5
Lat                                                     33.7932
Lng                                                     -84.388
Name: 0, dtype: object


In [8]:
# Test Return of No hotel found
hoteltest = hotelsearch('29.1081','21.2869')
print(hoteltest)

None


In [9]:
## Test Return of all results
hoteltest = hotelsearch(atlanta["lat"],atlanta["lng"], allresults=True)
print(hoteltest)

{'html_attributions': [], 'next_page_token': 'ATtYBwJGL51PVLjhuJ5PMKHu12s8IRtGOS9x_EO5LUo-z71h_XxnRMr3G5cOxltOYOWHGrJuZWJfPD6MFwAHLh3dDFAxRBCOI6R-yfS66CWtzvh02hEOWjPRUFKaJLzauXJvLxJWxdUX8Hynyq5lIiyXvxukn9P1LPptg9bxiN9DfSy6iKaQmNCBR-CeMHkFTBl5FFb2ENiFsqtSOR_DSjNdx1FMiGj8qZcn8gD9nuGtw6Fwtccj1KKtODvVapY6SK-YgpsceNuC5vd-MgIIoP527pX0FyXxvmnn0PBBU043Nh8grOV0U3kUm6CUVtxw13jLgWPxGpzQklS6GBtwXNGxMJdfeOQhZD0RKUxVH0VzQSzAB2dUt9-OA9iFRaGTzc7tg1ukUtZoK_zCbsYRSsliUhUUxoG7pIWhEll4NE084WFbGBsE32yHsCCCxJWU-lQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 33.758994, 'lng': -84.383111}, 'viewport': {'northeast': {'lat': 33.76047987989272, 'lng': -84.38204237010729}, 'southwest': {'lat': 33.75778022010728, 'lng': -84.38474202989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Sheraton Atlanta Hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1366, 'html_attributions': ['<a href="https://maps.google.c

In [10]:
# Object Returned

# icon Hotel_Name photos place_id rating Lat Lng

In [11]:
hotel_df = short_listdf.loc[:,['city','country','lat','lng']]
hotel_df['Hotel Name'] = ""
hotel_df.head()

,city,country,lat,lng,Hotel Name
127,Richards Bay,ZA,-28.7830,32.0377,
129,Santiago del Estero,AR,-27.7951,-64.2615,
217,Nador,MA,35.1740,-2.9287,
254,Zmiyiv,UA,49.6966,36.3611,
299,Yangi Marg`ilon,UZ,40.4272,71.7189,


In [12]:
hotel_df = short_listdf.loc[:,['city','country','lat','lng']]

search_counter = 0 
hotel_df['Hotel Name'] = ""
hotel_list = []

# Adding more details
hotel_df['hotel_place_id'] = ""
hotel_df['hotel_rating'] = ""
hotel_df['hotel_lat'] = ""
hotel_df['hotel_lng'] = ""
hotel_df['hotel_photos'] = ""

nitems = len(hotel_df)
for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    city_name = row["city"]
    search_counter = search_counter+1
    
    #Save the hotel name to the dataframe
    try:
        outtext = f"{search_counter} of {nitems} : Searching for hotels near {city_name}. \n"
        result=hotelsearch(lat,lng)
        if result is not None:
            hotel_list.append(result)
            hotelname = result['Hotel_Name']
            outtext = outtext + f"\t Found {hotelname}"
            hotel_df.loc[index, ["Hotel Name","hotel_place_id","hotel_rating","hotel_lat","hotel_lng","photos"]] = [result['Hotel_Name'], result['place_id'],result['rating'],result['Lat'],result['Lng'],result['photos']]
            print(outtext)
        else:
            outtext = outtext + f"\t Found {result}, removing...."
            hotel_df.drop(index)
            print(outtext)
    except (KeyError, IndexError) as e:
        print(f"Something Happend : {e}")
        #time.sleep(1)

# Need to clean up   
hotel_df = hotel_df.dropna()

c:\users\210057028\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


1 of 10 : Searching for hotels near Richards Bay. 
	 Found Aristo Manor
2 of 10 : Searching for hotels near Santiago del Estero. 
	 Found Plaza 3 Apart
3 of 10 : Searching for hotels near Nador. 
	 Found HOTEL MASTER HOME
4 of 10 : Searching for hotels near Zmiyiv. 
	 Found None, removing....
5 of 10 : Searching for hotels near Yangi Marg`ilon. 
	 Found Minimalist
6 of 10 : Searching for hotels near Afipskiy. 
	 Found None, removing....
7 of 10 : Searching for hotels near Murgab. 
	 Found None, removing....
8 of 10 : Searching for hotels near Siavonga. 
	 Found kariba inns
9 of 10 : Searching for hotels near Havelock. 
	 Found Holiday Inn Express & Suites Havelock NW-New Bern, an IHG Hotel
10 of 10 : Searching for hotels near Manakara. 
	 Found Hotel Antemoro


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
<dt>Rating</dt><dd>{hotel_rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["hotel_lat", "hotel_lng"]]
hotel_df[["hotel_lat", "hotel_lng"]]

,hotel_lat,hotel_lng
127,-28.7403,32.0764
129,-27.793,-64.2509
217,35.1491,-2.90979
299,40.3804,71.7767
487,-16.5323,28.7111
500,34.892,-76.9264
532,-22.1584,48.0169


In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))